### 3. SOYNLP : 비지도 학습으로 사용하는 단어 토크나이저

In [1]:
!pip install soynlp
import urllib.request
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor

In [2]:
# 훈련 데이터를 다수의 문서로 분리
corpus = DoublespaceLineCorpus("yes24Review.csv")
len(corpus)

3286

In [3]:
word_extractor = WordExtractor()
word_extractor.train(corpus)
word_score_table = word_extractor.extract()

training was done. used memory 0.115 Gb
all cohesion probabilities was computed. # words = 5331
all branching entropies was computed # words = 10339
all accessor variety was computed # words = 10339


In [4]:
print(word_score_table)

{'브': Scores(cohesion_forward=0, cohesion_backward=0, left_branching_entropy=1.0549201679861442, right_branching_entropy=0.47413931305783735, left_accessor_variety=3, right_accessor_variety=2, leftside_frequency=17, rightside_frequency=0), '(': Scores(cohesion_forward=0, cohesion_backward=0, left_branching_entropy=1.6460931106711414, right_branching_entropy=1.277034259466139, left_accessor_variety=7, right_accessor_variety=4, leftside_frequency=68, rightside_frequency=0), '않': Scores(cohesion_forward=0, cohesion_backward=0, left_branching_entropy=0.46354858027140006, right_branching_entropy=2.1535654786123377, left_accessor_variety=8, right_accessor_variety=12, leftside_frequency=370, rightside_frequency=0), '별': Scores(cohesion_forward=0, cohesion_backward=0, left_branching_entropy=2.5226820449772878, right_branching_entropy=1.3809803269389722, left_accessor_variety=14, right_accessor_variety=6, leftside_frequency=53, rightside_frequency=16), '순': Scores(cohesion_forward=0, cohesion_b

#### SOYNLP의 브랜칭 엔트로피(branching entropy) : 주어진 문자열에서 얼마나 다음 문자가 등장할 수 있는지를 판단하는 척도
- 

In [5]:
print(word_score_table["완"].right_branching_entropy)
print(word_score_table["완전"].right_branching_entropy)
print(word_score_table["완전한"].right_branching_entropy)

1.0219933064083193
0.6365141682948128
-0.0


#### SOYNLP의 응집 확률(cohesion probability) : 내부 문자열(substring)이 얼마나 응집하여 자주 등장하는지를 판단하는 척도
- 응집 확률은 문자열을 문자 단위로 분리하여 내부 문자열을 만드는 과정에서 왼쪽부터 순서대로 문자를 추가하면서 각 문자열이 주어졌을 때 그 다음 문자가 나올 확률을 계산하여 누적곱을 한 값입니다. 이 값이 높을수록 전체 코퍼스에서 이 문자열 시퀀스는 하나의 단어로 등장할 가능성이 높습니다

In [6]:
print(word_score_table["정유정"].cohesion_forward)
print(word_score_table["완전"].cohesion_forward)
print(word_score_table["완전한"].cohesion_forward)

0.40259099679869137
0.6103896103896104
0.5345224838248488


#### SOYNLP의 L tokenizer : 주로 L 토큰 + R 토큰의 형식을 갖는 어절 토큰

In [7]:
from soynlp.tokenizer import LTokenizer

scores = {word:score.cohesion_forward for word, score in word_score_table.items()}
l_tokenizer = LTokenizer(scores=scores)
l_tokenizer.tokenize("이 책에서 황혼을 맞이하는 방법으로 철학자들이 공통으로 말하는 것이 있다", flatten=False)

[('이', ''),
 ('책에서', ''),
 ('황혼을', ''),
 ('맞이', '하는'),
 ('방법', '으로'),
 ('철학', '자들이'),
 ('공통으로', ''),
 ('말하는', ''),
 ('것이', ''),
 ('있다', '')]

#### 최대 점수 토크나이저 : 띄어쓰기가 되지 않는 문장에서 점수가 높은 글자 시퀀스를 순차적으로 찾아내는 토크나이저

In [8]:
from soynlp.tokenizer import MaxScoreTokenizer

maxscore_tokenizer = MaxScoreTokenizer(scores=scores)
maxscore_tokenizer.tokenize("이 책에서 황혼을 맞이하는 방법으로 철학자들이 공통으로 말하는 것이 있다")

['이',
 '책에서',
 '황혼을',
 '맞이',
 '하는',
 '방법',
 '으로',
 '철학',
 '자들이',
 '공통으로',
 '말하는',
 '것이',
 '있다']

###  4. SOYNLP를 이용한 반복되는 문자 정제

In [9]:
from soynlp.normalizer import *

In [10]:
print(emoticon_normalize('앜ㅋㅋㅋㅋ개노잼ㅠㅠㅠㅠㅠ', num_repeats=1))
print(emoticon_normalize('앜ㅋㅋㅋㅋㅋㅋㅋㅋㅋ개노잼ㅠㅠㅠㅠ', num_repeats=3))
#불필요하게 반복되는 것은 하나로 정규화

아ㅋ노잼ㅠ
아ㅋㅋㅋ노잼ㅠㅠㅠ
